In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import bisect
import datetime
import csv

In [2]:
id_to_param_name = {
    1: "Дебит жидкости (объёмный), м3/сут", # target # Debit
    2: "Обводненность (объёмная), %",# Obvodnennost
    3: "Рбуф, атм",# Rbuf
    4: "Давление линейное, атм",# Linear pressure
    5: "Давление на приеме насоса, атм", # empty# Pressure na prieme nasosa
    6: "Загрузка ПЭД, %",# Zagruzka PED
    7: "Попутный газ",#  Poputniy gaz
    8: "Частота тока, Герц",# Chastota toka
    9: "Ток фазы А, A (ампер)",# Tok fazi
    10: "Мощность активная, кВт",# Moshnost aktivnaya
    11: "Напряжение, АВ Вольт",# Napryazhenie
    12: "P затрубное, атм", # empty# R zatrubnoye
}

In [3]:
def date_to_timestamp(x):
    return datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timestamp()

In [4]:
def dataframe_date_to_timestamp(x_list):
    y_list = []
    for x in x_list:
        y_list.append(datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timestamp())
    return y_list

In [13]:
def return_organized_entries(data):
    data.sort_values('tm_time')
    target = data[data['param_id'] == 1] # filter target data

    parameters_timeseries_per_well_timestamps = []
    parameters_timeseries_per_well_values = []
    for i in range(1, 13):
        parameters_timeseries_per_well_timestamps.append(dataframe_date_to_timestamp(data[data['param_id'] == i]['tm_time'].tolist())) # filter features, tm_value
        parameters_timeseries_per_well_values.append(data[data['param_id'] == i]['tm_value'].tolist())

    entries_for_well = []

    for i in range(target.shape[0]):
        entry = []
        entry.append(well_id)
        timestamp = date_to_timestamp(target.iloc[i].tm_time)
        entry.append(timestamp)
        id = 0
        for parameter_timestamps in parameters_timeseries_per_well_timestamps:
            #print(id)
            #if(len(parameter_timestamps)>1 and parameter_timestamps[0] <= timestamp and parameter_timestamps[-1] >= timestamp):
            if(len(parameter_timestamps)>0):
                index = bisect.bisect_left(parameter_timestamps, timestamp)
                #print(index)
                if index < 0:
                    entry.append(parameters_timeseries_per_well_values[id][index])
                elif len(parameter_timestamps)-index <= 1:
                    entry.append(parameters_timeseries_per_well_values[id][index-1])
                else:
                    entry.append((parameters_timeseries_per_well_values[id][index]+parameters_timeseries_per_well_values[id][index+1])*
                                 (parameter_timestamps[index]-timestamp)/parameter_timestamps[index]-parameter_timestamps[index+1])
                #id_to_param_name[i]
            else:
                entry.append(NaN)
            id += 1 #id for id_to_parameter should be id+1
        entries_for_well.append(entry)

    #df = pd.DataFrame(entries_for_well,
    #           columns =['timestamp', 'Debit', 'Obvodnennost', 'Rbuf', 'Linear pressure', 'Pressure na prieme nasosa', 'Zagruzka PED', 'Poputniy gaz', 
    #                     'Chastota toka', 'Tok fazi', 'Moshnost aktivnaya', 'Napryazhenie', 'R zatrubnoye', 'Well ID'])
    return entries_for_well

In [15]:
#all_entries = [] #createDataFrame(columns =['timestamp', 'Debit', 'Obvodnennost', 'Rbuf', 'Linear pressure', 'Pressure na prieme nasosa', 'Zagruzka PED', 
                                        #'Poputniy gaz', 'Chastota toka', 'Tok fazi', 'Moshnost aktivnaya', 'Napryazhenie', 'R zatrubnoye', 'Well ID'])

fields = ["Время","Дебит жидкости (объёмный), м3/сут","Обводненность (объёмная), %","Рбуф, атм","Давление линейное, атм","Давление на приеме насоса, атм",
          "Загрузка ПЭД, %","Попутный газ","Частота тока, Герц","Ток фазы А, A (ампер)","Мощность активная, кВт","Напряжение, АВ Вольт",
          "P затрубное, атм", "ID скважины"]


with open('Dynamic_data.csv', 'w') as f:
     
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerow(fields)

f.close()

for i in range(1,60):
    if i < 10:
        well_id = '1000'+ str(i)
    else:
        well_id = '100'+ str(i)
        
    data = pd.read_csv('train_dataset/tm_data/train_'+well_id+'.csv')

    with open('Dynamic_data.csv', 'a') as f:
        write = csv.writer(f)
        write.writerows(return_organized_entries(data))
    f.close()
    print(well_id)
    #all_entries.extend(return_organized_entries(data))

for i in range(1,40):
    if i < 10:
        well_id = '2000'+ str(i)
    else:
        well_id = '200'+ str(i)
        
    data = pd.read_csv('train_dataset/tm_data/train_'+well_id+'.csv')

    with open('Dynamic_data.csv', 'a') as f:
        write = csv.writer(f)
        write.writerows(return_organized_entries(data))
    f.close()
    print(well_id)
    #all_entries.extend(return_organized_entries(data))

10001
10002
10003


Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq\\backend\\cython\\checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
# Assuming df is your DataFrame
#sns.set(style="whitegrid")  # Setting the style to whitegrid for a clean background

plt.figure(figsize=(12, 6))  # Setting the figure size
sns.scatterplot(data=target, x='tm_time', y='tm_value', label='Дебит жидкости (объёмный), м3/сут', color='blue')
#sns.lineplot(data=df1[0:4000], x='tm_time', y='tm_value', label='Дебит жидкости (объёмный), м3/сут', color='blue')

# Adding labels and title
plt.xticks(rotation=90)
plt.xlabel('Время')
plt.ylabel('Дебит жидкости (объёмный), м3/сут')
plt.title('График')

plt.show()

In [ ]:
# # Assuming df is your DataFrame
# #sns.set(style="whitegrid")  # Setting the style to whitegrid for a clean background

# plt.figure(figsize=(12, 6))  # Setting the figure size
# sns.lineplot(data=df, x='tm_time', y='tm_value', label='Дебит жидкости (объёмный), м3/сут', color='blue')

# # Adding labels and title
# plt.xticks(rotation=90)
# plt.xlabel('Время')
# plt.ylabel('Дебит жидкости (объёмный), м3/сут')
# plt.title('График')

# plt.show()